In [ ]:
!pip install torch torchvision torchaudio norse opencv-python numpy tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s e

In [ ]:
!pip install norse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for norse: filename=norse-1.1.0-py3-none-any.whl size=1539019 sha256=eceb244a200aeb8d8f0ee0dbadfcc29725fc1648d115f3fd9b0dc0e045872bb3
  Stored in directory: /root/.cache/pip/wheels/2f/aa/7b/73285c582f07c47ad755bcaa0b8e18c13ace00e819e0c87c78
Successfully built norse


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/Avenue Dataset"
TRAIN_PATH = os.path.join(BASE_PATH, "training_videos")
TEST_PATH = os.path.join(BASE_PATH, "testing_videos")
OUTPUT_PATH = "/content/processed_frames"
TRAIN_OUTPUT = os.path.join(OUTPUT_PATH, "train")
TEST_OUTPUT = os.path.join(OUTPUT_PATH, "test")

os.makedirs(TRAIN_OUTPUT, exist_ok=True)
os.makedirs(TEST_OUTPUT, exist_ok=True)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def extract_frames(video_path, save_dir):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = transform(frame)
        torch.save(frame, os.path.join(save_dir, f"frame_{frame_count}.pt"))
        frame_count += 1
    cap.release()

def process_dataset(dataset_path, output_path):
    video_files = [f for f in os.listdir(dataset_path) if f.endswith(".avi")]
    for video in tqdm(video_files, desc=f"Processing {os.path.basename(dataset_path)} Videos"):
        video_path = os.path.join(dataset_path, video)
        save_dir = os.path.join(output_path, video.split('.')[0])
        os.makedirs(save_dir, exist_ok=True)
        extract_frames(video_path, save_dir)

process_dataset(TRAIN_PATH, TRAIN_OUTPUT)
process_dataset(TEST_PATH, TEST_OUTPUT)

print("✅ Data preprocessing complete! Train and test frames saved as PyTorch tensors.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing testing_videos Videos: 100%|██████████| 21/21 [02:09<00:00,  6.15s/it]

✅ Data preprocessing complete! Train and test frames saved as PyTorch tensors.


In [ ]:
import torch
import os
from torch.utils.data import Dataset, DataLoader

TRAIN_OUTPUT = "/content/processed_frames/train"
TEST_OUTPUT = "/content/processed_frames/test"

FIXED_FRAMES = 240

class AvenueDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.video_folders = sorted(os.listdir(data_path))

    def __len__(self):
        return len(self.video_folders)

    def __getitem__(self, index):
        video_folder = self.video_folders[index]
        frames_path = os.path.join(self.data_path, video_folder)

        frame_files = sorted(os.listdir(frames_path), key=lambda x: int(x.split('_')[-1].split('.')[0]))
        frames = [torch.load(os.path.join(frames_path, f)) for f in frame_files]

        num_frames = len(frames)
        if num_frames > FIXED_FRAMES:
            frames = frames[:FIXED_FRAMES]
        else:
            pad_frames = [torch.zeros_like(frames[0])] * (FIXED_FRAMES - num_frames)
            frames.extend(pad_frames)

        return torch.stack(frames)  # Shape: (T, C, H, W)

batch_size = 2
train_dataset = AvenueDataset(TRAIN_OUTPUT)
test_dataset = AvenueDataset(TEST_OUTPUT)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"✅ DataLoader Ready! {len(train_dataset)} training and {len(test_dataset)} testing videos loaded.")


✅ DataLoader Ready! 16 training and 21 testing videos loaded.


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import norse.torch as norse

class CNN_SNN_Model(nn.Module):
    def __init__(self):
        super(CNN_SNN_Model, self).__init__()

        resnet = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layer

        self.snn = norse.LIFRecurrentCell(512, 256)

        self.fc = nn.Linear(256, 1)  # Binary classification

    def forward(self, x):
        batch_size, T, C, H, W = x.shape
        x = x.view(-1, C, H, W)  # Merge batch & time for CNN input
        x = self.cnn(x)
        x = x.view(batch_size, T, -1)

        snn_out = []
        state = None
        for t in range(T):
            out, state = self.snn(x[:, t, :], state)
            snn_out.append(out)

        x = torch.stack(snn_out, dim=1)
        x = x.mean(dim=1)
        x = self.fc(x)
        return torch.sigmoid(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_SNN_Model().to(device)

print("✅ Hybrid CNN-SNN Model Ready! Running on:", device)


/usr/local/lib/python3.11/dist-packages/torch/utils/_cxx_pytree.py:238: UserWarning: PyTree type <class 'norse.torch.utils.pytree.LIFParameters'> is a subclass of `collections.namedtuple`, which is already registered in the global namespace. Override it with custom flatten/unflatten functions in namespace 'torch'.
  optree.register_pytree_node(
/usr/local/lib/python3.11/dist-packages/torch/utils/_cxx_pytree.py:238: UserWarning: PyTree type <class 'norse.torch.utils.pytree.LIFBoxParameters'> is a subclass of `collections.namedtuple`, which is already registered in the global namespace. Override it with custom flatten/unflatten functions in namespace 'torch'.
  optree.register_pytree_node(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarni

✅ Hybrid CNN-SNN Model Ready! Running on: cuda


In [ ]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

def train_model(model, train_loader, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        correct, total = 0, 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in loop:
            batch = batch.to(device)
            labels = torch.ones(batch.size(0), 1).to(device)

            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(loss=loss.item(), accuracy=correct / total)

        scheduler.step()
        print(f"🔥 Epoch [{epoch+1}/{epochs}] - Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

train_model(model, train_loader, epochs=20)
print("✅ Training Completed! Model is now trained on Avenue Dataset.")


Epoch 1/20:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_pa

🔥 Epoch [1/20] - Loss: 0.1832, Accuracy: 1.0000


Epoch 2/20:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_pa

🔥 Epoch [2/20] - Loss: 0.0120, Accuracy: 1.0000


Epoch 3/20:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_pa

...

🔥 Epoch [18/20] - Loss: 0.0007, Accuracy: 1.0000


Epoch 19/20:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_p

🔥 Epoch [19/20] - Loss: 0.0007, Accuracy: 1.0000


Epoch 20/20:   0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_p

🔥 Epoch [20/20] - Loss: 0.0007, Accuracy: 1.0000
✅ Training Completed! Model is now trained on Avenue Dataset.


In [ ]:
import torch
from tqdm import tqdm

def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    predictions, ground_truths = [], []

    with torch.no_grad():
        loop = tqdm(test_loader, desc="Evaluating")
        for batch in loop:
            batch = batch.to(device)
            labels = torch.ones(batch.size(0), 1).to(device)

            outputs = model(batch)
            preds = (outputs > 0.5).float()

            predictions.extend(preds.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(accuracy=correct / total)

    print(f"✅ Evaluation Complete! Accuracy: {correct / total:.4f}")

evaluate_model(model, test_loader)


Evaluating:   0%|          | 0/11 [00:00<?, ?it/s]<ipython-input-5-02f534f98279>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  frames = [torch.load(os.path.join(frames_p

✅ Evaluation Complete! Accuracy: 1.0000


In [ ]:
torch.save(model.state_dict(), "cnn_snn_model.pth")
print("✅ Model Saved Successfully!")

✅ Model Saved Successfully!


In [ ]:
model.load_state_dict(torch.load("cnn_snn_model.pth", map_location=device))
model.to(device)
model.eval()
print("✅ Model Loaded Successfully!")

<ipython-input-11-0044fa3eb838>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("cnn_snn_model.pth", map_location=device))


✅ Model Loaded Successfully!


In [ ]:
import cv2
import numpy as np

def predict_anomaly(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = transform(frame)
        frames.append(frame)

    cap.release()

    video_tensor = torch.stack(frames).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(video_tensor)
        prediction = "Anomaly" if output.item() > 0.5 else "Normal"

    print(f"🚨 Prediction: {prediction} (Confidence: {output.item():.4f})")

predict_anomaly("/content/Screen Recording 2025-01-05 164714.mp4", model)


🚨 Prediction: Anomaly (Confidence: 0.9993)
